## AB Tests

### 1. Оценка результатов AB теста

В данном задании вам требуется оценить результаты AB теста. Даны результаты на контрольной и тестовой группе после проведения AB теста.
Помимо показателей целевой переменной, вам даны также несколько фичей, которые являются характеристиками объектов. Даны целевые показатели (y) и принадлежность к целевой группе (W).

</u>Задача:</u>
1. Проверьте подходящую для данных гипотезу о равенстве средних в выборках
2. Каков treatment effect нашего воздействия? (ATE) 
3. Примените известные вам методы оценки ATE с учетом особенностей данных.
4. Проанализируйте, сравните результаты применения различных методов.
5. В явном виде выпишите выводы.

</i>Примечание: предположения о рандомизированном эксперименте нуждаются в проверке - независимость treatment от объектов не гарантирована. Все предположения выписывайте явно.</i> 

Для данной задачи используется dataset:  dataset_part1_students.csv

In [143]:
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import ztest
from scipy.stats import chisquare
from sklearn.linear_model import LinearRegression, LogisticRegression
import statsmodels.formula.api as smf
import numpy as np
import causalml

In [111]:
dataset1 = pd.read_csv("data\dataset_part1_students.csv", sep='\t')

In [112]:
dataset1.head()

,y,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,-1.459404,0.0,0.986277,0.873392,0.509746,0.271836,0.336919,0.216954,0.276477,0.343316,0.862159,0.156700
1,-1.694174,0.0,0.140887,0.757080,0.736325,0.355663,0.341093,0.666803,0.217101,0.561427,0.124179,0.319736
2,5.378374,0.0,0.953214,0.137357,0.569413,0.975665,0.503367,0.667664,0.034191,0.456119,0.155851,0.476049
3,2.188189,1.0,0.169702,0.896258,0.373394,0.379693,0.858317,0.646061,0.583462,0.668350,0.177793,0.849248
4,2.862696,1.0,0.442373,0.831468,0.763921,0.919691,0.070573,0.156165,0.636894,0.555696,0.191929,0.425656


#### 1. Гипотеза о равенстве средних в выборках

In [113]:
y0 = dataset1[dataset1['W'] == 0]['y'].array
y1 = dataset1[dataset1['W'] == 1]['y'].array

In [114]:
ztest(y1, y0)

(9.332400850259313, 1.0349941796938945e-20)

Здесь pvalue - очень маленькое значение, значит гипотеза о равенстве средних отвергается

#### 2,3. Treatment effect

Чтобы найти ATE, пусть y - это Y, так как в ином случае только по таргету y мы не сможем найти ATE. Но тогда исходя из 1 пункта у нас уже получается зависимость Y от W, потому что средние значения y отличаются в обоих случаях. Это можно увидеть и отсюда:

In [115]:
dataset1.groupby('W').mean()

,y,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
W,,,,,,,,,,,
0.0,1.866349,0.39691,0.390728,0.505678,0.499297,0.497471,0.499891,0.496947,0.497261,0.499343,0.499442
1.0,2.266452,0.59806,0.593573,0.497915,0.502627,0.497835,0.502462,0.498642,0.496682,0.500562,0.498393


Отсюда также очевидно, что если характеристики X_3 - X_10 в обоих случаях (при W = 0 и W = 1) независимы и их средние примерно равны, то вот средние значения X_1 и X_2 в обоих пулах сильно отличаются. Это значит, что в пул W = 0 и в пул W = 1 по этим критериям объекты отбирались не рандомизированно.

Можно использовать linear regression для нахождения ATE (это будет коэффициент перед W):

In [116]:
model = LinearRegression()
x = dataset1.drop('y', axis=1)
y = dataset1['y']
model.fit(x, y)
ATE_1 = model.coef_[0]

In [117]:
ATE_1

0.5133854882565507

#### 4. Анализ и сравнение результатов

Здесь можно попробовать сравнить наше значение ATE, полученное в линейной регрессии, с разницей средних (это значение было бы, если бы W и Y у нас были бы полностью независимы). Они должны отличаться, что мы и видим:

In [118]:
Y_0 = dataset1.groupby('W').mean()['y'][0]
Y_1 = dataset1.groupby('W').mean()['y'][1]
ATE_if_independent = Y_1 - Y_0

In [119]:
ATE_if_independent

0.4001031055444413

#### 5. Выводы

Если принять, что в данном случае у нас всё же дан Y, то можно рассчитать ATE, используя линейную регрессию, так как W и Y зависимы. Если же это было бы не так, мы бы нашли ATE как разность Y_1 и Y_0

### 2. Оценка результатов AB теста с показателями до начала эксперимента

Необходимо как и ранее оценить результаты AB теста, а именно ATE. Вы располагаете результатами до и после проведения теста (y_0 и y_1), а также вам дан treatment assignment (W). 

Необходимо: 
1. Проверить все требуемые гипотезы
2. Оценить ATE подходящим для этого случая методом (примените несколько методов)
3. Сравнить и проанализировать результаты оценок
4. Явно выписать выводы.

Как и прежде, все предположения должны быть указаны. Визуализация результатов приветствуется.

dataset_part2_students.csv

In [120]:
dataset2 = pd.read_csv("data\dataset_part2_students.csv", sep='\t')

In [121]:
dataset2.head()

,y_1,y_0,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,5.887156,2.785060,1.0,0.605978,0.733369,0.138947,0.312673,0.997243,0.128162,0.178993,0.752925,0.662161,0.784310
1,2.232601,5.533515,0.0,0.096894,0.058571,0.962396,0.616557,0.086630,0.561272,0.616525,0.963843,0.574304,0.371161
2,0.144419,0.095546,1.0,0.452145,0.201850,0.569305,0.195096,0.583704,0.476313,0.517814,0.823099,0.732225,0.069056
3,-2.446425,3.590472,1.0,0.672129,0.643485,0.828014,0.204469,0.617489,0.617701,0.301069,0.871741,0.589654,0.981770
4,2.163090,1.365405,0.0,0.442232,0.126318,0.508831,0.431786,0.915940,0.709016,0.890655,0.588886,0.636830,0.342209


#### 1. Гипотезы

In [122]:
dataset2['Y'] = dataset2['y_1'] - dataset2['y_0']

In [123]:
dataset2.head()

,y_1,y_0,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,Y
0,5.887156,2.785060,1.0,0.605978,0.733369,0.138947,0.312673,0.997243,0.128162,0.178993,0.752925,0.662161,0.784310,3.102097
1,2.232601,5.533515,0.0,0.096894,0.058571,0.962396,0.616557,0.086630,0.561272,0.616525,0.963843,0.574304,0.371161,-3.300914
2,0.144419,0.095546,1.0,0.452145,0.201850,0.569305,0.195096,0.583704,0.476313,0.517814,0.823099,0.732225,0.069056,0.048874
3,-2.446425,3.590472,1.0,0.672129,0.643485,0.828014,0.204469,0.617489,0.617701,0.301069,0.871741,0.589654,0.981770,-6.036897
4,2.163090,1.365405,0.0,0.442232,0.126318,0.508831,0.431786,0.915940,0.709016,0.890655,0.588886,0.636830,0.342209,0.797684


Проверим гипотезу о равных средних снова через Z тест:

In [124]:
Y0 = dataset2[dataset2['W'] == 0]['Y'].array
Y1 = dataset2[dataset2['W'] == 1]['Y'].array

In [125]:
ztest(Y1,Y0)

(9.118025905018952, 7.650513264824386e-20)

Z тест снова и в этом случае показывает, что treatment зависит от target. Это снова видно отсюда (X_1, X_2 и Y для случаев W = 0 and W = 1):

In [126]:
dataset2.groupby('W').mean()

,y_1,y_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,Y
W,,,,,,,,,,,,,
0.0,1.878542,0.816187,0.387120,0.400934,0.495181,0.502530,0.497533,0.499039,0.504281,0.503380,0.501267,0.498942,1.062355
1.0,2.309793,0.692534,0.592249,0.597398,0.501800,0.497781,0.501576,0.494910,0.501227,0.497676,0.503586,0.497798,1.617259


#### 2. Оценка ATE

##### 1.Для начала снова попробуем применить linear regression:

In [127]:
ATE_2LR

0.5425071531831982

##### 2. DiD

In [128]:
did = smf.ols(formula='Y ~ W', data=dataset2).fit()
did.params[1]

0.5549047702971286

##### 3. CUPED

In [129]:
theta = np.cov((dataset2['y_1'], dataset2['y_0'])/np.var(dataset2['y_0']))[0][1]
theta

0.00449360408176727

In [130]:
t_cuped = dataset2['y_1'] - (dataset2['y_0'] - dataset2['y_0'].mean())*theta

In [131]:
modelLR3 = LinearRegression()
xLR3 = dataset2.drop(['y_1', 'y_0', 'Y'], axis=1)
yLR3 = t_cuped
modelLR3.fit(xLR3, yLR3)
ATE_3LR = modelLR3.coef_[0]

In [132]:
ATE_3LR

0.5481679819724223

#### 3. Выводы 

Все методы показали примерно одинаковый результат с небольшими отличиями

### 3. Построение uplift модели по результатам AB теста.

Вам даны результаты AB теста для модели конверсии клиента в продукте (conversion). Конверсия может принимать значения 1 или 0, что значит - клиент подключил услугу, либо не подключил соответственно.

Задача:
1. Оцените ATE по данным.
2. Оцените CATE для каждого наблюдения.
3. Проставьте рекомендации к воздействию на каждый объект.
4. Какая ожидаемая конверсия при оптимальном воздействии?
5. Все выводы, предпосылки и методы представляйте в явном виде в ноутбуке.

Визуализация результатов также будет приветствоваться.

dataset_part3_students.csv

In [193]:
dataset3 = pd.read_csv("data\dataset_part3_students.csv", sep='\t')

In [194]:
dataset3.head()

,treatment,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,conversion
0,control,0.034351,0.550725,-1.348524,-0.029591,0.499535,-0.412464,0.933588,-0.522954,-0.029591,...,-0.314573,-0.879456,0.640264,1.165722,0.570495,0.529443,0.227962,1.929914,0.798664,0
1,control,-1.059404,0.299404,0.220273,-1.569660,1.084204,-2.167118,1.216469,-0.744547,-1.569660,...,0.303664,1.080643,2.646483,-0.329690,1.725636,1.434649,-2.064893,0.188000,-1.134651,0
2,control,0.051934,2.338729,-0.341017,-1.099362,0.693090,-2.424267,1.572066,-0.212798,-1.099362,...,-0.083721,-1.981712,0.224777,2.038004,2.516398,2.324280,-3.182196,-0.888490,-1.402561,0
3,treatment,0.667770,0.539316,0.468719,-0.840885,-0.708331,0.126634,1.275854,-0.954919,-0.840885,...,-0.752011,-0.347319,-1.659402,-0.871980,-0.893513,-0.658078,-2.704997,-0.390745,-1.578004,1
4,control,-1.416158,2.486628,0.496402,-1.174618,2.738267,-4.756755,-0.180918,-1.281432,-1.174618,...,-1.198151,-0.232961,0.948875,1.210523,1.536723,1.548610,-0.926435,0.972459,-0.711251,0


In [195]:
transform_dict = {'control': 0, 'treatment': 1}
dataset3['treatment'] = dataset3['treatment'].transform(lambda x: transform_dict.get(x))
dataset3

,treatment,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,conversion
0,0,0.034351,0.550725,-1.348524,-0.029591,0.499535,-0.412464,0.933588,-0.522954,-0.029591,...,-0.314573,-0.879456,0.640264,1.165722,0.570495,0.529443,0.227962,1.929914,0.798664,0
1,0,-1.059404,0.299404,0.220273,-1.569660,1.084204,-2.167118,1.216469,-0.744547,-1.569660,...,0.303664,1.080643,2.646483,-0.329690,1.725636,1.434649,-2.064893,0.188000,-1.134651,0
2,0,0.051934,2.338729,-0.341017,-1.099362,0.693090,-2.424267,1.572066,-0.212798,-1.099362,...,-0.083721,-1.981712,0.224777,2.038004,2.516398,2.324280,-3.182196,-0.888490,-1.402561,0
3,1,0.667770,0.539316,0.468719,-0.840885,-0.708331,0.126634,1.275854,-0.954919,-0.840885,...,-0.752011,-0.347319,-1.659402,-0.871980,-0.893513,-0.658078,-2.704997,-0.390745,-1.578004,1
4,0,-1.416158,2.486628,0.496402,-1.174618,2.738267,-4.756755,-0.180918,-1.281432,-1.174618,...,-1.198151,-0.232961,0.948875,1.210523,1.536723,1.548610,-0.926435,0.972459,-0.711251,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,-1.309691,2.925832,-0.584101,-1.232931,2.601140,-4.943288,0.416758,-0.083626,-1.232931,...,1.041399,0.975982,0.636279,1.073711,-0.406603,0.027760,-0.478887,1.265746,0.059865,0
199996,0,-1.131361,1.562317,1.265630,-1.361879,1.169695,-3.247293,1.253718,-1.968727,-1.361879,...,-0.485884,0.674360,0.892121,0.893261,0.182528,0.338611,1.006028,-1.943045,-0.145377,0
199997,0,0.442433,2.202153,-0.380019,-0.474935,0.766306,-1.722354,0.751083,0.595800,-0.474935,...,1.098074,0.299918,0.718669,1.031289,0.398560,0.591750,1.620529,-1.081292,0.995816,0
199998,1,-0.084879,0.423688,0.389931,-1.996510,-0.123098,-1.103408,2.207197,0.402468,-1.996510,...,-0.511203,-0.150816,-0.256026,2.231563,0.848577,0.739235,1.656343,0.436962,0.622010,0


#### 1. Оценка ATE

In [255]:
model_up = LinearRegression()
x_up = dataset3.drop('conversion', axis=1)
y_up = dataset3['conversion']
model_up.fit(x_up, y_up)
ATE_up = model_up.coef_[0]
print(ATE_up)

0.06024388343541732


#### 2. CATE

#### S-learner

1. Строим 1 модуль на всех данные
2. Делаем предикт, если бы для всех данных W был бы 0, и второй как будто бы W для всех 1
3. CATE вычисляем как разность векторов с 1 и 0

In [169]:
from sklearn.ensemble import RandomForestClassifier

In [214]:
clf = RandomForestClassifier(max_depth=7)
clf.fit(xLR_3, yLR_3)
nuF_w0 = clf.predict_proba(x_w0)[:, 0]
nuF_w1 = clf.predict_proba(x_w1)[:, 0]

In [226]:
print(nuF_w0)
print(nuF_w1)

[0.93259929 0.94402752 0.95040757 ... 0.95432838 0.94966312 0.94866374]
[0.93384363 0.93868517 0.94555569 ... 0.96209385 0.95359431 0.95270346]


In [223]:
CATE_S = nuF_w1 - nuF_w0

Для примера можно привести максимальное воздействие на объект:

In [224]:
CATE_S.max()

0.34119836582060564

In [229]:
CATE_S_mean = nuF_w1.mean() - nuF_w0.mean()
CATE_S_mean

-0.03404859747111655

##### T-learner:

1. Берём все данные и строим по ним 2 модели: одну для treatment = 0 и другую для treatment = 1
2. Берём полный сет данных (уже с 0 и 1) и подаём на предикт в обе модели
3. CATE вычисляем как разность векторов с 1 и 0

In [246]:
data_for_w0 = dataset3[dataset3['treatment'] == 0]
data_for_w1 = dataset3[dataset3['treatment'] == 1]
x_for_w0 = data_for_w0.drop(['treatment', 'conversion'], axis=1)
x_for_w1 = data_for_w1.drop(['treatment', 'conversion'], axis=1)
y_for_w0 = data_for_w0['conversion']
y_for_w1 = data_for_w1['conversion']

In [247]:
m_for_w0 = RandomForestClassifier(max_depth=7)
m_for_w1 = RandomForestClassifier(max_depth=7)
m_for_w0.fit(x_for_w0, y_for_w0)
m_for_w1.fit(x_for_w1, y_for_w1)

RandomForestClassifier(max_depth=7)

Теперь возьмём данные для теста:

In [248]:
x_test = dataset3.drop(['treatment', 'conversion'], axis=1)

In [249]:
nuT_w0 = m_for_w0.predict_proba(x_test)[:, 0]
nuT_w1 = m_for_w1.predict_proba(x_test)[:, 0]

In [250]:
print(nuT_w0)
print(nuT_w1)

[0.93938078 0.9316886  0.9599331  ... 0.95709822 0.95820076 0.95192153]
[0.93137567 0.92857329 0.9301306  ... 0.98377249 0.96110992 0.96106831]


In [251]:
CATE_T = nuT_w1 - nuT_w0

In [252]:
CATE_T_mean = nuT_w1.mean() - nuT_w0.mean()

In [256]:
CATE_T

array([-0.00800511, -0.00311531, -0.0298025 , ...,  0.02667428,
        0.00290916,  0.00914678])

In [253]:
CATE_T_mean

-0.05947475924125878

#### 3. Простановка рекомендаций

Будем считать, что recommendation = 1 (рекомендуем), если CATE не отрицательно && conversion != 1 (потому что таким объектам уже нет смысла что-то рекомендовать) и recommendation = 0 в противном случае:

In [267]:
dataset3['CATE'] = CATE_T
inversed_conversion = dataset3['conversion'].apply(lambda x: 1 if x == 0 else 0)
#Здесь следующим шагом делаю так, чтобы лямбда ниже выдавала верные рекомендации
dataset3['recommendation'] = dataset3['CATE'] * inversed_conversion
dataset3['recommendation'] = dataset3['recommendation'].apply(lambda x: 0 if x <= 0 else 1)

In [268]:
dataset3

,treatment,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_16,X_17,X_18,X_19,X_20,X_21,X_22,conversion,CATE,recommendation
0,0,0.034351,0.550725,-1.348524,-0.029591,0.499535,-0.412464,0.933588,-0.522954,-0.029591,...,0.640264,1.165722,0.570495,0.529443,0.227962,1.929914,0.798664,0,-0.008005,0
1,0,-1.059404,0.299404,0.220273,-1.569660,1.084204,-2.167118,1.216469,-0.744547,-1.569660,...,2.646483,-0.329690,1.725636,1.434649,-2.064893,0.188000,-1.134651,0,-0.003115,0
2,0,0.051934,2.338729,-0.341017,-1.099362,0.693090,-2.424267,1.572066,-0.212798,-1.099362,...,0.224777,2.038004,2.516398,2.324280,-3.182196,-0.888490,-1.402561,0,-0.029802,0
3,1,0.667770,0.539316,0.468719,-0.840885,-0.708331,0.126634,1.275854,-0.954919,-0.840885,...,-1.659402,-0.871980,-0.893513,-0.658078,-2.704997,-0.390745,-1.578004,1,-0.770145,0
4,0,-1.416158,2.486628,0.496402,-1.174618,2.738267,-4.756755,-0.180918,-1.281432,-1.174618,...,0.948875,1.210523,1.536723,1.548610,-0.926435,0.972459,-0.711251,0,-0.022194,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,-1.309691,2.925832,-0.584101,-1.232931,2.601140,-4.943288,0.416758,-0.083626,-1.232931,...,0.636279,1.073711,-0.406603,0.027760,-0.478887,1.265746,0.059865,0,-0.054218,0
199996,0,-1.131361,1.562317,1.265630,-1.361879,1.169695,-3.247293,1.253718,-1.968727,-1.361879,...,0.892121,0.893261,0.182528,0.338611,1.006028,-1.943045,-0.145377,0,0.016102,1
199997,0,0.442433,2.202153,-0.380019,-0.474935,0.766306,-1.722354,0.751083,0.595800,-0.474935,...,0.718669,1.031289,0.398560,0.591750,1.620529,-1.081292,0.995816,0,0.026674,1
199998,1,-0.084879,0.423688,0.389931,-1.996510,-0.123098,-1.103408,2.207197,0.402468,-1.996510,...,-0.256026,2.231563,0.848577,0.739235,1.656343,0.436962,0.622010,0,0.002909,1


#### 4. Ожидаемая конверсия

Если рассматривать конверсию в данном случае как число объектов, на которые мы воздействовали, то это число нужно отнести к количеству 0 в графе conversion до воздействия чтобы получить процент "обработанных" объектов:

In [276]:
numerator = dataset3['recommendation'].value_counts()[1]
denom = dataset3['conversion'].value_counts()[0]

In [278]:
conversion_level = numerator/denom * 100
print(conversion_level)

29.372008051330184
